In [31]:
# Brightway packages
import bw2io as bi
import bw2data as bd
import pandas as pd
# Brightway type hints
from bw2io import SingleOutputEcospold2Importer
# import/export
import os

In [32]:
from bw2data import projects, databases
from bw2io import create_default_biosphere3

projects.set_current("PB_LCIA")

if "biosphere3" not in databases:
    create_default_biosphere3()
    print("biosphere3 created.")
else:
    print("biosphere3 already exists.")

print("Databases now:", list(databases))


biosphere3 already exists.
Databases now: ['biosphere3']


In [33]:
print(bd.projects)
print(bd.projects.current)
print(bd.databases)

Brightway2 projects manager with 3 objects:
	LCA_EPE
	PB_LCIA
	default
Use `projects.report()` to get a report on all projects.
PB_LCIA
Databases dictionary with 1 object(s):
	biosphere3


In [34]:
bio = bd.Database("biosphere3")

rows = []
for flow in bio:
    # Each flow is a dict-like dataset
    rows.append({
        "name": flow.get("name"),
        "categories": " / ".join(flow.get("categories", [])) if flow.get("categories") else None,
        "unit": flow.get("unit"),
        "type": flow.get("type"),          # e.g., "emission", "resource", etc.
        "location": flow.get("location"),  # often None for biosphere flows
        "database": flow.get("database"),  # should be "biosphere3"
        "code": flow.get("code"),          # unique identifier inside the database
        "reference_product": flow.get("reference product"),
        "comment": flow.get("comment"),
        "CAS": flow.get("CAS number"),
        "formula": flow.get("formula"),
    })

df = pd.DataFrame(rows)

# Optional: sort for readability
df = df.sort_values(["categories", "name"], na_position="last").reset_index(drop=True)

# 3) Write to Excel
output_path = "biosphere3_export.xlsx"
df.to_excel(output_path, index=False)

print(f"Exported {len(df)} biosphere flows to: {output_path}")

Exported 4709 biosphere flows to: biosphere3_export.xlsx
